In [1]:
import onnx
import transformers,torch
from transformers import AutoTokenizer,AutoModelForCausalLM
from optimum.onnxruntime import ORTModelForCausalLM
from peft import PeftModelForCausalLM
from transformers.onnx import FeaturesManager

/root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.7/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /root/anaconda3/envs/baichuan7B did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [2]:
model_path="../../ptm/baichuan/"

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)


In [4]:
model=AutoModelForCausalLM.from_pretrained(model_path,trust_remote_code=True)

In [20]:
tokenizer=AutoTokenizer.from_pretrained("/data/bert-family/bert-base-chinese/")

In [12]:
import os
from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)
from transformers.data.processors.squad import SquadV1Processor
total_samples = 1000

# ONNX opset version
opset_version=11
cache_dir = "./squad"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")


In [5]:
text="你好，我最近心情不太好，你可以唱首歌吗"

In [6]:

inp=tokenizer(text,return_tensors="pt")
inp

{'input_ids': tensor([[ 9875, 31213,    72, 31182,  6769, 11582, 13204, 31213,    72, 12620,
         32481, 31723, 32065, 31763]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
pseudal_input={
    "input_ids":inp["input_ids"],
    "attention_mask":inp["attention_mask"]
}

In [21]:
# Registering custom operation for unflatten
@onnxscript.script(custom_opset)
def aten_unflatten(self, dim, sizes):
    """unflatten(Tensor(a) self, int dim, SymInt[] sizes) -> Tensor(a)"""

    self_size = op.Shape(self)

    if dim < 0:
        # PyTorch accepts negative dim as reversed counting
        self_rank = op.Size(self_size)
        dim = self_rank + dim

    head_start_idx = op.Constant(value_ints=[0])
    head_end_idx = op.Reshape(dim, op.Constant(value_ints=[1]))
    head_part_rank = op.Slice(self_size, head_start_idx, head_end_idx)

    tail_start_idx = op.Reshape(dim + 1, op.Constant(value_ints=[1]))
    #tail_end_idx = op.Constant(value_ints=[_INT64_MAX])
    tail_end_idx = op.Constant(value_ints=[9223372036854775807]) # = sys.maxint, exactly 2^63 - 1 -> 64 bit int
    tail_part_rank = op.Slice(self_size, tail_start_idx, tail_end_idx)

    final_shape = op.Concat(head_part_rank, sizes, tail_part_rank, axis=0)

    return op.Reshape(self, final_shape)

def custom_unflatten(g, self, dim, shape):
    return g.onnxscript_op(aten_unflatten, self, dim, shape).setType(self.type().with_sizes([1, 2, 3, 4, 5]))  # For example, you know size would be 1,2,3,4,5

torch.onnx.register_custom_op_symbolic(
    symbolic_name="aten::unflatten",
    symbolic_fn=custom_unflatten,
    opset_version=18,
)

In [22]:
# load config
with torch.no_grad():
    symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
    torch.onnx.export(model=model,
        args=tuple(pseudal_input.values()),
        f="./baichuan.onnx",
        opset_version=18,                      # the ONNX version to export the model to
        do_constant_folding=True,
        input_names=['input_ids',                         # the model's input names
                    'input_mask'],
        output_names=["hidden_states","presents"],
        custom_opsets = {"torch.onnx": 18},
        dynamic_axes={
            "input_ids":symbolic_names,
            "attention_mask":symbolic_names
        }
        
    )

/root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/torch/onnx/utils.py:2029: UserWarning: Provided key attention_mask for dynamic axes is not a valid input/output name
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/modeling_baichuan.py:375: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1:
/root/.cache/huggingface/modules/transformers_modules/modeling_baichuan.py:46: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask = torch.full((tgt_len, tgt_len), torch.tensor(

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 # load config                                                                               │
│    2 with torch.no_grad():                                                                       │
│    3 │   symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}                                    │
│ ❱  4 │   torch.onnx.export(model=model,                                                          │
│    5 │   │   args=tuple(pseudal_input.values()),                                                 │
│    6 │   │   f="./baichuan.onnx",                                                                │
│    7 │   │   opset_version=18,                      # the ONNX version to export the model to    │
│                                                                                                  │
│ /root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/torch/onnx/utils.py:506 in export    │
│                                                                                                  │
│    503 │   │   │   All errors are subclasses of :class:`errors.OnnxExporterError`.               │
│    504 │   """                                                                                   │
│    505 │                                                                                         │
│ ❱  506 │   _export(                                                                              │
│    507 │   │   model,                                                                            │
│    508 │   │   args,                                                                             │
│    509 │   │   f,                                                                                │
│                                                                                                  │
│ /root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/torch/onnx/utils.py:1548 in _export  │
│                                                                                                  │
│   1545 │   │   │   │   dynamic_axes = {}                                                         │
│   1546 │   │   │   _validate_dynamic_axes(dynamic_axes, model, input_names, output_names)        │
│   1547 │   │   │                                                                                 │
│ ❱ 1548 │   │   │   graph, params_dict, torch_out = _model_to_graph(                              │
│   1549 │   │   │   │   model,                                                                    │
│   1550 │   │   │   │   args,                                                                     │
│   1551 │   │   │   │   verbose,                                                                  │
│                                                                                                  │
│ /root/anaconda3/envs/baichuan7B/lib/python3.9/site-packages/torch/onnx/utils.py:1117 in          │
│ _model_to_graph                                                                                  │
│                                                                                                  │
│   1114 │   params_dict = _get_named_param_dict(graph, params)                                    │
│   1115 │                                                                                         │
│   1116 │   try:                                                                                  │
│ ❱ 1117 │   │   graph = _optimize_graph(                                                          │
│   1118 │   │   │   graph,                                                                        │
│   1119 │   │   │   operator_export_type,                   